In [13]:
import torch
import torch.utils.data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [33]:
tweet_data = pd.read_csv("text_quality.csv", encoding = "ISO-8859-1")
tweet_data = tweet_data.drop_duplicates(subset="text")


# Feature Selection
features = ['no_chars', 'no_words', 'no_unique_words', 'word_difference', 'number_hashtags' 'number_mentions', 'number_urls',
           'listed_count', 'statuses_count', 'followers_count', 'favourites_count', 'friends_count']

reduced_tweet_data = pd.read_csv("text_quality.csv", encoding = "ISO-8859-1", names = features, low_memory = False)

reduced_tweet_data.head(5)
#tweet_data.head(5)

,,,,,,,no_chars,no_words,no_unique_words,word_difference,number_hashtagsnumber_mentions,number_urls,listed_count,statuses_count,followers_count,favourites_count,friends_count
text,no_chars,no_words,no_unique_words,word_difference,number_hashtags,number_mentions,number_urls,client_sources,listed_count,verified,geo_enabled,statuses_count,followers_count,favourites_count,user_lang,friends_count,class
#DanAndPhilBRITs Make sure they zoom in on Louis during Little Mix performance #BRITs2016,4.49980967,2.63905733,2.63905733,0,1.098612289,0,0,Twitter for iPhone,4,FALSE,TRUE,5604,259,4880,nl,446,reg
I voted for DRAG ME DOWN by @onedirection for British Video at the #BRITs 2016 https://t.co/C9Rw6W3F4v ##BRITVIDONEDIRECTION,4.828313737,2.890371758,2.833213344,0.693147181,1.098612289,0.693147181,0.693147181,Mobile Web,0,FALSE,FALSE,198,64,370,en,380,cont
I voted for DRAG ME DOWN by @onedirection for British Video at the #BRITs 2016 https://t.co/zWB24WLWSi ##BRITVIDONEDIRECTION x22,4.86753445,2.995732274,2.944438979,0.693147181,1.098612289,0.693147181,0.693147181,Twitter Web Client,4,FALSE,FALSE,3843,794,2401,en,334,cont
@brits @danisnotonfire @AmazingPhil #BRITs #DanAndPhilBRITs South America,4.304065093,2.079441542,2.079441542,0,1.098612289,1.386294361,0,Twitter for iPhone,9,FALSE,TRUE,24432,3608,29325,en,2974,cont


In [20]:
inputData = tweet_data.drop(["text","client_sources","user_lang","class"], axis=1).as_matrix() #np.float32()
outputData = tweet_data["class"].as_matrix()

# trainloader_input = torch.utils.data.DataLoader(inputData, batch_size=4, shuffle=True, num_workers=8)
# trainloader_output = torch.utils.data.DataLoader(outputData, batch_size=4, shuffle=True, num_workers=8)
# print(trainloader_output)

In [21]:
# Example NN using torch

from torch.autograd import Variable
import torch.nn as nn

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 1, inputData.shape[1], 8, 2

# Set our variables up
x_data = inputData # Variable(torch.from_numpy(inputData))
y_data = outputData # Variable(torch.from_numpy(outputData), requires_grad=False)


class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred


model = TwoLayerNet(D_in, H, D_out)

In [278]:
# Optimize and error calculation

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for t in range(1):
    # Forward pass: Compute predicted y by passing x to the model
    for i, data in enumerate(zip(x_data, y_data), 0):
    #for i, data in enumerate(zip(trainloader_input, trainloader_output), 0):
        
        x, y = data
#         print(y)
#         x = Variable(x)
#         y = Variable(torch.stack(y, 0))
#         x = np.long32(x).tolist()
        
#         x = Variable(torch.from_numpy(x))
        
        x = Variable(torch.FloatTensor([x]))
        y = Variable(torch.IntTensor([int(y)])) #[int(y)]))
        #y = Variable(torch.FloatTensor(y)) #[int(y)]))
        
        y_pred = model(x)
        
#         print(x)
#         print(y.long())
#         print(y_pred)

        # Compute and print loss
        # .long().unsqueeze(0)
        loss = criterion(y_pred, y.long())
        print(t, loss.data[0])

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # print statistics
#         running_loss += loss.data[0]
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 2000))
#             running_loss = 0.0

0 0.7492404580116272
0 0.6405326128005981
0 0.6405341029167175
0 0.6400880813598633
0 0.6392402052879333
0 0.7514780163764954
0 0.7521370053291321
0 0.7521713376045227
0 0.7516434192657471
0 0.7506104111671448
0 0.6401376724243164
0 0.7482873797416687
0 0.642065167427063
0 0.7463023066520691
0 0.6437242031097412
0 0.7445939779281616
0 0.645162045955658
0 0.7431125044822693
0 0.646417498588562
0 0.6467363238334656
0 0.7420005202293396
0 0.7416167259216309
0 0.647732138633728
0 0.6480099558830261
0 0.7406431436538696
0 0.7402979731559753
0 0.6489023566246033
0 0.7391682267189026
0 0.7383774518966675
0 0.7371215224266052
0 0.6525622606277466
0 0.6534876227378845
0 0.7340405583381653
0 0.7331363558769226
0 0.7317837476730347
0 0.6575764417648315
0 0.7289527654647827
0 0.6599830389022827
0 0.6607807278633118
0 0.7263287901878357
0 0.7255550622940063
0 0.7243273258209229
0 0.7226930856704712
0 0.666337251663208
0 0.7194002270698547
0 0.6691731214523315
0 0.6701454520225525
0 0.67054408788681

0 0.8438351154327393
0 0.842090904712677
0 0.5652185082435608
0 0.8383759260177612
0 0.5678802132606506
0 0.5688751339912415
0 0.8344061374664307
0 0.833156406879425
0 0.8313941955566406
0 0.5734277367591858
0 0.5745937824249268
0 0.5752623081207275
0 0.8265277147293091
0 0.576171338558197
0 0.8253369331359863
0 0.8244314789772034
0 0.5782447457313538
0 0.8221806883811951
0 0.5799387693405151
0 0.580508291721344
0 0.5806331634521484
0 0.5803576111793518
0 0.5797222256660461
0 0.8223241567611694
0 0.5783950090408325
0 0.8237122297286987
0 0.8239079117774963
0 0.5778793096542358
0 0.5778135061264038
0 0.8241065144538879
0 0.5774618983268738
0 0.5771605372428894
0 0.8252136707305908
0 0.8253385424613953
0 0.8248195052146912
0 0.8237220048904419
0 0.5789353251457214
0 0.8211451172828674
0 0.8196543455123901
0 0.5824122428894043
0 0.5834199786186218
0 0.5839369297027588
0 0.5840110182762146
0 0.8160791993141174
0 0.8158252835273743
0 0.5845626592636108
0 0.584778368473053
0 0.58458054065704

0 0.40586185455322266
0 0.4045479893684387
0 0.4031476080417633
0 1.1062438488006592
0 0.40078771114349365
0 0.3997763693332672
0 0.39865121245384216
0 0.3974250555038452
0 0.39610958099365234
0 0.3947153091430664
0 0.39325177669525146
0 1.126666784286499
0 0.3907969892024994
0 0.38975217938423157
0 0.38860586285591125
0 0.38736966252326965
0 0.38605406880378723
0 0.38466861844062805
0 0.383221834897995
0 0.38172146677970886
0 0.38017451763153076
0 0.3785873055458069
0 1.158170461654663
0 1.1604130268096924
0 1.161489725112915
0 0.37543785572052
0 0.3752315640449524
0 1.1628057956695557
0 0.37493711709976196
0 1.1628730297088623
0 1.1621602773666382
0 0.3758664131164551
0 0.3763209879398346
0 0.3765336871147156
0 1.1591260433197021
0 0.37695416808128357
0 0.37714046239852905
0 0.3771107494831085
0 0.37688663601875305
0 0.37648800015449524
0 0.37593284249305725
0 0.37523767352104187
0 0.3744174540042877
0 0.373485803604126
0 1.168092966079712
0 1.1691968441009521
0 0.3719377815723419
0 

0 0.27282869815826416
0 0.2726629376411438
0 1.433593988418579
0 1.4331895112991333
0 0.27300429344177246
0 0.2733205258846283
0 0.27349093556404114
0 0.27352985739707947
0 0.2734503149986267
0 0.2732643485069275
0 0.27298277616500854
0 0.27261558175086975
0 0.2721717059612274
0 1.4359629154205322
0 0.27156129479408264
0 0.27135998010635376
0 0.27106600999832153
0 0.2706890404224396
0 1.4405306577682495
0 0.27019354701042175
0 0.2700415551662445
0 0.2697928845882416
0 1.4430497884750366
0 0.2695167064666748
0 1.443047046661377
0 0.26976683735847473
0 1.4415143728256226
0 1.4398669004440308
0 0.271266907453537
0 0.2718963623046875
0 0.2723504304885864
0 0.27264586091041565
0 0.27279800176620483
0 0.27282091975212097
0 1.4325482845306396
0 1.4316571950912476
0 1.4296975135803223
0 0.2745435833930969
0 0.2752586901187897
0 0.27578768134117126
0 0.2761482000350952
0 1.4210618734359741
0 0.2769107520580292
0 0.27729326486587524
0 0.27752047777175903
0 1.417141318321228
0 1.4157485961914062


0 0.2870895266532898
0 1.3884111642837524
0 0.28725120425224304
0 0.28737300634384155
0 1.3872674703598022
0 0.2877192497253418
0 0.28791970014572144
0 0.2879749834537506
0 0.28789952397346497
0 0.28770649433135986
0 0.28740793466567993
0 1.3882988691329956
0 0.28703567385673523
0 0.28692999482154846
0 0.2867105007171631
0 0.28638890385627747
0 0.285975843667984
0 0.28548097610473633
0 1.3946477174758911
0 0.28477534651756287
0 1.3958147764205933
0 0.2846793830394745
0 0.28469231724739075
0 0.2845812141895294
0 1.396331548690796
0 0.2845308780670166
0 0.2845633625984192
0 0.2844699025154114
0 0.2842632234096527
0 0.28395503759384155
0 0.283555805683136
0 0.28307515382766724
0 0.2825217843055725
0 0.28190362453460693
0 0.28122779726982117
0 0.2805008292198181
0 0.27972856163978577
0 0.27891626954078674
0 0.27806878089904785
0 0.2771903872489929
0 0.27628499269485474
0 0.27535608410835266
0 0.27440688014030457
0 0.2734401822090149
0 0.2724585235118866
0 0.2714642286300659
0 1.43981707096

...

0 0.24919743835926056
0 0.2484024316072464
0 0.24759235978126526
0 0.24676933884620667
0 0.245935320854187
0 0.24509203433990479
0 0.24424102902412415
0 0.24338370561599731
0 1.5354766845703125
0 0.24208472669124603
0 0.24160005152225494
0 0.2410726100206375
0 1.5428498983383179
0 0.24033501744270325
0 0.24008895456790924
0 1.5455412864685059
0 1.545339822769165
0 0.2402162104845047
0 0.24047182500362396
0 0.24061065912246704
0 1.5423462390899658
0 0.2410108596086502
0 0.2412494719028473
0 0.24137243628501892
0 0.2413911521434784
0 0.241316020488739
0 0.24115653336048126
0 0.240921288728714
0 0.24061815440654755
0 0.2402542680501938
0 1.545322060585022
0 0.2397955358028412
0 0.23966817557811737
0 1.5467004776000977
0 0.23961330950260162
0 0.23965799808502197
0 0.23960739374160767
0 0.23947110772132874
0 0.23925787210464478
0 0.2389756143093109
0 1.549778699874878
0 0.23865658044815063
0 0.23858892917633057
0 0.2384379804134369
0 0.23821227252483368
0 0.23791952431201935
0 1.55373942852

0 0.24401384592056274
0 0.24371397495269775
0 0.24335087835788727
0 0.2429313212633133
0 0.24246138334274292
0 0.2419465333223343
0 1.5396029949188232
0 1.5401955842971802
0 0.24142144620418549
0 0.24150170385837555
0 0.24148190021514893
0 0.2413720041513443
0 1.540374755859375
0 0.24134644865989685
0 0.24140319228172302
0 0.24136224389076233
0 0.2412334829568863
0 0.24102582037448883
0 0.2407473623752594
0 0.2404055893421173
0 0.24000711739063263
0 0.23955807089805603
0 0.23906394839286804
0 0.238529771566391
0 0.23796001076698303
0 0.23735876381397247
0 0.23672965168952942
0 0.23607610166072845
0 0.23540098965168
0 0.23470714688301086
0 0.2339969128370285
0 0.2332724630832672
0 0.23253585398197174
0 0.2317887842655182
0 0.23103289306163788
0 0.23026956617832184
0 0.22950012981891632
0 0.22872576117515564
0 1.5904499292373657
0 1.591914415359497
0 1.5919640064239502
0 0.22787316143512726
0 0.22807209193706512
0 1.5895887613296509
0 0.22857947647571564
0 1.586867332458496
0 0.229451492

0 0.34204918146133423
0 1.2388635873794556
0 0.3425320088863373
0 0.3427649438381195
0 0.34280624985694885
0 0.34267497062683105
0 0.3423886001110077
0 0.3419630527496338
0 0.34141287207603455
0 1.2421448230743408
0 0.34056711196899414
0 0.3402349352836609
0 1.2445658445358276
0 1.244585633277893
0 0.3401651084423065
0 0.34036189317703247
0 0.34037259221076965
0 0.3402157723903656
0 0.3399084210395813
0 0.3394661247730255
0 0.33890292048454285
0 0.33823174238204956
0 0.33746418356895447
0 0.3366108536720276
0 1.2547427415847778
0 0.3352537751197815
0 1.257189154624939
0 1.257401704788208
0 0.3349538743495941
0 0.3350900113582611
0 0.33505046367645264
0 0.33485278487205505
0 0.3345131278038025
0 0.3340461552143097
0 0.3334652781486511
0 0.3327826261520386
0 0.33200928568840027
0 0.3311552405357361
0 0.33022958040237427
0 0.3292405307292938
0 1.2737594842910767
0 0.3276596665382385
0 0.32702213525772095
0 0.3262939155101776
0 0.325484961271286
0 0.3246043026447296
0 0.32366010546684265
0

0 1.0715404748916626
0 1.0679675340652466
0 1.063897728919983
0 0.4256705343723297
0 1.0557875633239746
0 0.4297701418399811
0 1.048493504524231
0 1.0447648763656616
0 1.0405769348144531
0 1.035981297492981
0 1.0310248136520386
0 1.0257500410079956
0 0.4471069574356079
0 0.449666827917099
0 0.4517173171043396
0 0.4533040523529053
0 1.0072678327560425
0 0.45598530769348145
0 0.45708411931991577
0 1.0014965534210205
0 0.9995784759521484
0 0.9970563650131226
0 0.46218690276145935
0 0.9917089939117432
0 0.4652133584022522
0 0.9867749810218811
0 0.984110951423645
0 0.9809339046478271
0 0.47212719917297363
0 0.4738176465034485
0 0.9724622964859009
0 0.9698539972305298
0 0.47855323553085327
0 0.4799838364124298
0 0.48098301887512207
0 0.9618092179298401
0 0.4826129674911499
0 0.9591506719589233
0 0.48427820205688477
0 0.484919011592865
0 0.9560028314590454
0 0.48592817783355713
0 0.9542659521102905
0 0.48708420991897583
0 0.9523243308067322
0 0.9509676694869995
0 0.48960205912590027
0 0.94769

0 0.7010387182235718
0 0.6993908882141113
0 0.6974047422409058
0 0.6951173543930054
0 0.6937324404716492
0 0.6907672882080078
0 0.6886578798294067
0 0.6862687468528748
0 0.7027541995048523
0 0.6817630529403687
0 0.6795967817306519
0 0.6771658658981323
0 0.6745004653930664
0 0.7151398062705994
0 0.6695483922958374
0 0.719779908657074
0 0.7214744687080383
0 0.6646569967269897
0 0.7238724231719971
0 0.6626513004302979
0 0.7257596850395203
0 0.7262693643569946
0 0.6611565947532654
0 0.726627767086029
0 0.6608861088752747
0 0.7268538475036621
0 0.6607275605201721
0 0.6604290008544922
0 0.7277589440345764
0 0.7279328107833862
0 0.6598843932151794
0 0.7277136445045471
0 0.7273223400115967
0 0.7264364361763
0 0.6621772646903992
0 0.7244110107421875
0 0.7232541441917419
0 0.6654015183448792
0 0.7207731604576111
0 0.7194263935089111
0 0.7176902294158936
0 0.6711812615394592
0 0.6724961996078491
0 0.673201322555542
0 0.6733561754226685
0 0.7136926054954529
0 0.6732085347175598
0 0.713810324668884

In [198]:
# -*- coding: utf-8 -*-
import torch
from torch.autograd import Variable


class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred


# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 1, 1000, 100, 2

# Create random Tensors to hold inputs and outputs, and wrap them in Variables
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# Construct our model by instantiating the class defined above
model = TwoLayerNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(1):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)
    
    print(x)
    print("Y: ",y)
    print("YPRED: ", y_pred)

    # Compute and print loss
    loss = criterion(y_pred, y)
    print(t, loss.data[0])

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Variable containing:
-0.7783  1.1923 -0.4279  ...  -1.4640  0.8462 -0.0376
[torch.FloatTensor of size 1x1000]

Y:  Variable containing:
-0.8173  0.1845
[torch.FloatTensor of size 1x2]

YPRED:  Variable containing:
 0.0973  0.2042
[torch.FloatTensor of size 1x2]

0 0.8369063138961792


In [171]:
torch.FloatTensor([x])

RuntimeError: already counted a million dimensions in a given sequence. Most likely your items are also sequences and there's no way to infer how many dimension should the tensor have